# Evaluating Hyperparameters Part 2: Deep Learning

### This script contains the following:

#### 1. Importing Libraries and Data
#### 2. Data Wrangling
#### 3. Reshaping for Modeling
#### 4. Data Split
#### 5. Bayesian Hyperparameter Optimization
#### 6. Running CNN with Optimized Search Parameters
#### 7. Confusion Matrix

### 1. Importing Libraries and Data

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
from sklearn.model_selection import cross_val_score
from numpy import unique
from numpy import reshape
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
leaky_relu = LeakyReLU(alpha=0.1)
import warnings

In [5]:
# Define path
path = r'C:\Users\16307\Desktop\CareerFoundry\Tasks - Machine Learning\ClimateWins Analysis\Data Sets'

In [6]:
# Import cleaned data
df = pd.read_csv(os.path.join(path, 'unscaled_cleaned_with_date.csv'), index_col = False)
df

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,19600102,1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,19600103,1,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,19600104,1,3,0.92,1.0180,0.58,0.00,4.1,6.3,3.8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,19600105,1,6,0.95,1.0180,0.65,0.14,5.4,3.0,-0.7,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,20221027,10,1,0.79,1.0248,1.34,0.22,7.7,15.9,11.4,...,14.2,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22946,20221028,10,6,0.77,1.0244,1.34,0.22,5.4,16.7,14.3,...,14.3,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22947,20221029,10,4,0.76,1.0227,1.34,0.22,6.1,16.7,13.1,...,14.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22948,20221030,10,5,0.80,1.0212,1.34,0.22,5.8,15.4,11.6,...,12.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5


In [7]:
# Read in the pleasant weather data
pleasant = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'), index_col = False)
pleasant

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,20221027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,20221028,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,20221029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,20221030,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2. Data Wrangling

In [9]:
# Drop unnecessary columns
df.drop(['DATE', 'MONTH'], axis=1, inplace=True)

In [10]:
df.shape

(22950, 135)

In [11]:
# Drop unnecessary columns
pleasant.drop(columns = 'DATE', inplace = True)

In [12]:
pleasant.shape

(22950, 15)

### 3. Reshaping for Modeling

In [14]:
# Turn df to arrays
X = np.array(df)
y = np.array(pleasant)

In [15]:
X = X.reshape(-1,15,9)

In [16]:
X.shape

(22950, 15, 9)

In [17]:
# Use argmax to transform y
y = np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [18]:
y.shape

(22950,)

In [19]:
# Check y layout
type_of_target(y)

'multiclass'

The Bayesian optimization function only accepts y data in “multiclass” layout.

### 4. Data Split 

In [22]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [23]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


### 5. Bayesian Hyperparameter Optimization

In [25]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [26]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [27]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\16307\anaconda3\envs\tf_env\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/25
15/15 - 2s - loss: 2.7075 - accuracy: 0.6036 - 2s/epoch - 158ms/step
Epoch 2/25
15/15 - 1s - loss: 2.7004 - accuracy: 0.6440 - 895ms/epoch - 60ms/step
Epoch 3/25
15/15 - 1s - loss: 2.6970 - accuracy: 0.6440 - 898ms/epoch - 60ms/step
Epoch 4/25
15/15 - 1s - loss: 2.6942 - accuracy: 0.6440 - 909ms/epoch - 61ms/step
Epoch 5/25
15/15 - 1s - loss: 2.6916 - accuracy: 0.6440 - 950ms/epoch - 63ms/step
Epoch 6/25
15/15 - 1s - loss: 2.6893 - accuracy: 0.6440 - 915ms/epoch - 61ms/step
Epoch 7/25
15/15 - 1s - loss: 2.6872 - accuracy: 0.6440 - 891ms/epoch - 59ms/step
Epoch 8/25
15/15 - 1s - loss: 2.6852 - accuracy: 0.6440 - 907ms/epoch - 60ms/step
Epoch 9/25
15/15 - 1s - loss: 2.6833 - accuracy: 0.6440 - 906ms/epoch - 60ms/step
Epoch 10/25
15/15 - 1s - loss: 2.6815 - accuracy: 0.6440 - 892ms/epoch - 59ms/step
Epoch 11/25
15/15 - 1s - loss: 2.6798 - accuracy: 0.6440 - 995ms/epoch - 66ms/step
Epoch 12/25
15/15 - 1s - loss: 2.6781 - accuracy: 0.6440 - 934ms/epoch - 62ms/step
Epoch 13/25
15/

C:\Users\16307\anaconda3\envs\tf_env\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/25
15/15 - 2s - loss: 2.7049 - accuracy: 0.5995 - 2s/epoch - 162ms/step
Epoch 2/25
15/15 - 1s - loss: 2.7005 - accuracy: 0.6440 - 927ms/epoch - 62ms/step
Epoch 3/25
15/15 - 1s - loss: 2.6971 - accuracy: 0.6440 - 955ms/epoch - 64ms/step
Epoch 4/25
15/15 - 1s - loss: 2.6943 - accuracy: 0.6440 - 929ms/epoch - 62ms/step
Epoch 5/25
15/15 - 1s - loss: 2.6918 - accuracy: 0.6440 - 908ms/epoch - 61ms/step
Epoch 6/25
15/15 - 1s - loss: 2.6895 - accuracy: 0.6440 - 932ms/epoch - 62ms/step
Epoch 7/25
15/15 - 1s - loss: 2.6874 - accuracy: 0.6440 - 891ms/epoch - 59ms/step
Epoch 8/25
15/15 - 1s - loss: 2.6855 - accuracy: 0.6440 - 914ms/epoch - 61ms/step
Epoch 9/25
15/15 - 1s - loss: 2.6836 - accuracy: 0.6440 - 911ms/epoch - 61ms/step
Epoch 10/25
15/15 - 1s - loss: 2.6818 - accuracy: 0.6440 - 892ms/epoch - 59ms/step
Epoch 11/25
15/15 - 1s - loss: 2.6802 - accuracy: 0.6440 - 939ms/epoch - 63ms/step
Epoch 12/25
15/15 - 1s - loss: 2.6786 - accuracy: 0.6440 - 925ms/epoch - 62ms/step
Epoch 13/25
15/

...

C:\Users\16307\anaconda3\envs\tf_env\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/35
21/21 - 2s - loss: 1.5754 - accuracy: 0.5524 - 2s/epoch - 111ms/step
Epoch 2/35
21/21 - 1s - loss: 1.0080 - accuracy: 0.6588 - 703ms/epoch - 33ms/step
Epoch 3/35
21/21 - 1s - loss: 0.9199 - accuracy: 0.6829 - 699ms/epoch - 33ms/step
Epoch 4/35
21/21 - 1s - loss: 0.8617 - accuracy: 0.6983 - 690ms/epoch - 33ms/step
Epoch 5/35
21/21 - 1s - loss: 0.8116 - accuracy: 0.7155 - 659ms/epoch - 31ms/step
Epoch 6/35
21/21 - 1s - loss: 0.7680 - accuracy: 0.7310 - 666ms/epoch - 32ms/step
Epoch 7/35
21/21 - 1s - loss: 0.7315 - accuracy: 0.7428 - 666ms/epoch - 32ms/step
Epoch 8/35
21/21 - 1s - loss: 0.6951 - accuracy: 0.7551 - 661ms/epoch - 31ms/step
Epoch 9/35
21/21 - 1s - loss: 0.6630 - accuracy: 0.7649 - 653ms/epoch - 31ms/step
Epoch 10/35
21/21 - 1s - loss: 0.6342 - accuracy: 0.7730 - 658ms/epoch - 31ms/step
Epoch 11/35
21/21 - 1s - loss: 0.6056 - accuracy: 0.7848 - 667ms/epoch - 32ms/step
Epoch 12/35
21/21 - 1s - loss: 0.5775 - accuracy: 0.7942 - 658ms/epoch - 31ms/step
Epoch 13/35
21/

C:\Users\16307\anaconda3\envs\tf_env\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


21/21 - 2s - loss: 1.6620 - accuracy: 0.5015 - 2s/epoch - 95ms/step
Epoch 2/35
21/21 - 1s - loss: 1.0150 - accuracy: 0.6501 - 641ms/epoch - 31ms/step
Epoch 3/35
21/21 - 1s - loss: 0.9177 - accuracy: 0.6810 - 656ms/epoch - 31ms/step
Epoch 4/35
21/21 - 1s - loss: 0.8492 - accuracy: 0.7051 - 654ms/epoch - 31ms/step
Epoch 5/35
21/21 - 1s - loss: 0.7928 - accuracy: 0.7242 - 646ms/epoch - 31ms/step
Epoch 6/35
21/21 - 1s - loss: 0.7486 - accuracy: 0.7361 - 662ms/epoch - 32ms/step
Epoch 7/35
21/21 - 1s - loss: 0.7099 - accuracy: 0.7476 - 650ms/epoch - 31ms/step
Epoch 8/35
21/21 - 1s - loss: 0.6737 - accuracy: 0.7627 - 667ms/epoch - 32ms/step
Epoch 9/35
21/21 - 1s - loss: 0.6423 - accuracy: 0.7744 - 662ms/epoch - 32ms/step
Epoch 10/35
21/21 - 1s - loss: 0.6093 - accuracy: 0.7855 - 658ms/epoch - 31ms/step
Epoch 11/35
21/21 - 1s - loss: 0.5791 - accuracy: 0.7977 - 678ms/epoch - 32ms/step
Epoch 12/35
21/21 - 1s - loss: 0.5435 - accuracy: 0.8086 - 672ms/epoch - 32ms/step
Epoch 13/35
21/21 - 1s - lo

ValueError: Input y contains NaN.

In [47]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 460,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 47,
 'kernel': 1.9444298503238986,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x1e986e16c40>}

### 6. Running CNN with Optimized Search Parameters

In [49]:
# Set the model with optimized hyperparameters

epochs = 47
batch_size = 460

timesteps = len(X_train[0])  
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = int(round(1.9444298503238986))  
neurons = 61
normalization = 0.770967179954561  
dropout = 0.7296061783380641  
dropout_rate = 0.19126724140656393  
optimizer = Adadelta(learning_rate=0.7631771981307285)  

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

# Add second set of dense layers
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())  # Max-pooling layer to downsample the data
model.add(Flatten())  # Flatten for the dense layers
model.add(Dense(n_classes, activation='softmax'))  # Output layer for classification

# Compile the model with specified loss and optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [50]:
model.summary()

Model: "sequential_75"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_75 (Conv1D)          (None, 14, 61)            1159      
                                                                 
 batch_normalization_30 (Ba  (None, 14, 61)            244       
 tchNormalization)                                               
                                                                 
 dense_380 (Dense)           (None, 14, 61)            3782      
                                                                 
 dropout_45 (Dropout)        (None, 14, 61)            0         
                                                                 
 dense_381 (Dense)           (None, 14, 61)            3782      
                                                                 
 dense_382 (Dense)           (None, 14, 61)            3782      
                                                     

In [51]:
# Put the y_test set back into a one-hot configuration
y_train_one_hot = to_categorical(y_train, num_classes=15)

In [52]:
# Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [53]:
# Compile the model with categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [54]:
# Fit the model to the data
model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/47
38/38 - 3s - loss: 1.3554 - accuracy: 0.6057 - 3s/epoch - 67ms/step
Epoch 2/47
38/38 - 1s - loss: 0.8978 - accuracy: 0.6968 - 1s/epoch - 29ms/step
Epoch 3/47
38/38 - 1s - loss: 0.8069 - accuracy: 0.7218 - 1s/epoch - 29ms/step
Epoch 4/47
38/38 - 1s - loss: 0.7522 - accuracy: 0.7401 - 1s/epoch - 28ms/step
Epoch 5/47
38/38 - 1s - loss: 0.7245 - accuracy: 0.7506 - 1s/epoch - 29ms/step
Epoch 6/47
38/38 - 1s - loss: 0.6856 - accuracy: 0.7660 - 1s/epoch - 32ms/step
Epoch 7/47
38/38 - 1s - loss: 0.6700 - accuracy: 0.7713 - 1s/epoch - 28ms/step
Epoch 8/47
38/38 - 1s - loss: 0.6436 - accuracy: 0.7787 - 1s/epoch - 28ms/step
Epoch 9/47
38/38 - 1s - loss: 0.6268 - accuracy: 0.7833 - 1s/epoch - 28ms/step
Epoch 10/47
38/38 - 1s - loss: 0.6094 - accuracy: 0.7901 - 1s/epoch - 28ms/step
Epoch 11/47
38/38 - 1s - loss: 0.5873 - accuracy: 0.7978 - 1s/epoch - 28ms/step
Epoch 12/47
38/38 - 1s - loss: 0.5755 - accuracy: 0.8025 - 1s/epoch - 27ms/step
Epoch 13/47
38/38 - 1s - loss: 0.5578 - accuracy:

### 7. Confusion Matrix

In [56]:
# Define list of stations names
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [57]:
print("Unique classes in y_test:", np.unique(y_test))

Unique classes in y_test: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]


In [58]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [59]:
# Before making predictions, convert y_test to one-hot format
y_test_one_hot = to_categorical(y_test, num_classes=15)

In [60]:
# Before making predictions, convert y_test to one-hot format
y_test_one_hot = to_categorical(y_test, num_classes=15)

# Predict the class probabilities
y_pred = model.predict(X_test)

# Convert y_test and y_pred to class labels
if y_test_one_hot.ndim == 1:
    y_test_labels = y_test_one_hot
else:
    y_test_labels = np.argmax(y_test_one_hot, axis=1)

if y_pred.ndim == 1:
    y_pred_labels = y_pred
else:
    y_pred_labels = np.argmax(y_pred, axis=1)

# Manually calculate accuracy
correct_predictions = np.sum(y_test_labels == y_pred_labels)
total_samples = len(y_test_labels)
accuracy = correct_predictions / total_samples

print(f'Accuracy: {accuracy * 100:.2f}%')

180/180 [==============================] - 1s 2ms/step
Accuracy: 83.69%


In [61]:
y_pred_labels = np.argmax(y_pred, axis=1)

In [62]:
# Convert y_test_one_hot back to class labels
y_test_labels = np.argmax(y_test_one_hot, axis=1)

In [63]:
# Evaluate
print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3475       111         8       2           3         1       0   
BELGRADE      246       846         0       0           0         0       0   
BUDAPEST       53        63        97       1           0         0       0   
DEBILT         30         7        14      31           0         0       0   
DUSSELDORF     12         1         5       5           6         0       0   
HEATHROW       25         7         7       7           5        24       0   
KASSEL          2         3         1       0           1         0       2   
LJUBLJANA      22        10         7       0           0         4       0   
MAASTRICHT      8         0         0       0           0         0       0   
MADRID         58        36        29       7           3        12       1   
MUNCHENB        6         1         0       0       

In [64]:
print("Unique values in y_test:", np.unique(y_test_labels))
print("Unique values in y_pred:", np.unique(y_pred_labels))

Unique values in y_test: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
Unique values in y_pred: [ 0  1  2  3  4  5  6  7  8  9 11]
